In [24]:
import sys,os
import json
import requests
import numpy as np
import pandas as pd
from pystac_client import Client
cat_url = "http://localhost:8080"
cat_client = Client.open(cat_url)
intake_cat = "/home/aradick/Documents/spear-flp/catalog_blue.csv"

In [ ]:
def search(url: str, **kwargs):
    r = requests.get(url+"/search", params=search)
    return r.json["features"]

def get_paths(items):
    paths = []
    for x in items:
        for a in x["assets"].values():
            paths.append(a["href"])
    return paths

def save_paths(output_name, paths):
    df2 = pd.DataFrame({"source" : paths, "dest" : paths})
    df2.to_csv("my_search.txt", sep=' ', index=False, header=False)

In [34]:
def get_paths_requests(url: str, search: dict):
    paths = []
    r = requests.get(url+"/search", params=search)
    for x in r.json()["features"]:
        for a in x["assets"].values():
            paths.append(a["href"])
    return paths

def get_paths_pystac(cat: Client, **kwargs):
    paths = []
    s = cat.search(**kwargs)
    for i in s.items():
        for a in i.assets.values():
            paths.append(a.href)
    return paths

def get_paths_df(df, **kwargs):
    truths = np.ones(len(df.index), dtype=bool)
    for (k,v) in kwargs.items():
        truths &= (df[k] == v)
    return df.loc[truths, 'path']

def get_paths_intake(cat_path: str, **kwargs):
    df = pd.read_csv(cat_path)
    return get_paths_df(df, **kwargs)

In [21]:
%timeit get_paths_requests(cat_url, {'filter': "variable_id='snow'"})

24.6 ms ± 4.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%timeit get_paths_pystac(cat_client, filter="variable_id='snow'")

32.4 ms ± 2.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
cat_df = pd.read_csv("/home/aradick/Documents/spear-flp/catalog_blue.csv")
%timeit get_paths_df(cat_df, variable_id='snow')

443 μs ± 2.17 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [35]:
%timeit get_paths_intake(intake_cat, variable_id='snow')

14.5 ms ± 169 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
